In [ ]:
# Import necessary libraries

import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.interpolate import Rbf

# Create DataFrame with the provided data
data = {
    'Country': ['China', 'Japan', 'Liberia', 'South Korea', 'Marshall Islands', 
                'Saudi Arabia', 'Qatar', 'Bahamas', 'United Arab Emirates', 'Israel'],
    'Total_Spending': [446099458, 387265853, 353119848, 303178372, 284773327,
                       271797541, 250901327, 239466140, 225555192, 188886398],
    'Latitude': [35.8617, 36.2048, 6.4281, 35.9078, 7.1315, 
                 23.8859, 25.3548, 25.0343, 23.4241, 31.0461],
    'Longitude': [104.1954, 138.2529, -9.4295, 127.7669, 171.1845, 
                  45.0792, 51.1839, -77.3963, 53.8478, 34.8516]
}

df = pd.DataFrame(data)

# Convert spending to millions for better readability
df['Spending_Millions'] = df['Total_Spending'] / 1000000

# Create a mesh grid for the surface
resolution = 100
lon_range = np.linspace(-180, 180, resolution)
lat_range = np.linspace(-90, 90, resolution)
lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)

# Use Radial Basis Function interpolation to create a continuous 3D surface
# The epsilon parameter controls the "peakiness" of the elevation
rbf = Rbf(df['Longitude'], df['Latitude'], df['Spending_Millions'], function='multiquadric', epsilon=15)
spending_grid = rbf(lon_grid, lat_grid)

# Ensure no negative values in our elevation model
spending_grid = np.maximum(spending_grid, 0)

# Create the 3D surface plot
fig = go.Figure(data=[go.Surface(
    z=spending_grid,
    x=lon_grid,
    y=lat_grid,
    colorscale='Viridis',
    colorbar=dict(title='Lobbying Spending ($ millions)'),
    opacity=0.8
)])

# Add country markers with labels
fig.add_trace(go.Scatter3d(
    x=df['Longitude'],
    y=df['Latitude'],
    z=df['Spending_Millions'] + 20,  # Offset to ensure markers appear above the surface
    mode='markers+text',
    text=df['Country'],
    textposition="top center",
    marker=dict(
        size=10,
        color=df['Spending_Millions'],
        colorscale='Viridis',
        opacity=1
    ),
    hovertemplate='<b>%{text}</b><br>Spending: $%{z:.1f} million<extra></extra>'
))

# Update layout with appropriate camera angle and aspect ratio
fig.update_layout(
    title='Foreign Lobbying Spending by Country (3D Visualization)',
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Spending ($ millions)',
        aspectratio=dict(x=2, y=1, z=0.3),
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        )
    ),
    height=800,
    width=1000
)

fig.show()

In [ ]:
# Import necessary libraries
import plotly.graph_objects as go
import pandas as pd

# Create DataFrame with the provided data
lobbying_data = {
    'Country': ['China', 'Japan', 'Liberia', 'South Korea', 'Marshall Islands', 
                'Saudi Arabia', 'Qatar', 'Bahamas', 'United Arab Emirates', 'Israel'],
    'Total_Spending': [446099458, 387265853, 353119848, 303178372, 284773327,
                       271797541, 250901327, 239466140, 225555192, 188886398],
    # ISO3 country codes for mapping
    'iso_alpha': ['CHN', 'JPN', 'LBR', 'KOR', 'MHL', 
                  'SAU', 'QAT', 'BHS', 'ARE', 'ISR'],
    # Approximate central coordinates for each country
    'lat': [35.86, 36.20, 6.43, 35.91, 7.13,
            23.89, 25.35, 25.03, 23.42, 31.05],
    'lon': [104.20, 138.25, -9.43, 127.77, 171.18,
            45.08, 51.18, -77.40, 53.85, 34.85]
}

df = pd.DataFrame(lobbying_data)

# Convert spending to millions for better readability
df['Spending_Millions'] = df['Total_Spending'] / 1000000

# ----- PART 1: Interactive Flat World Map -----
# Create choropleth base map with enhanced interactivity
fig_map = go.Figure()

# Add the base choropleth layer
fig_map.add_trace(
    go.Choropleth(
        locations=df['iso_alpha'],
        z=df['Spending_Millions'],
        text=df['Country'],
        colorscale='Viridis',
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title='Spending<br>($ millions)',
        showscale=True,
        hovertemplate='<b>%{text}</b><br>Spending: $%{z:.1f} million<extra></extra>'
    )
)

# Add bubble markers proportional to spending for visual emphasis
fig_map.add_trace(
    go.Scattergeo(
        lon=df['lon'],
        lat=df['lat'],
        text=df['Country'],
        mode='markers+text',
        marker=dict(
            size=df['Spending_Millions']/5,  # Scaled size for better visualization
            color=df['Spending_Millions'],
            colorscale='Viridis',
            opacity=0.8,
            line=dict(width=0)
        ),
        textposition="top center",
        hovertemplate='<b>%{text}</b><br>Spending: $%{marker.color:.1f} million<extra></extra>'
    )
)

# Configure the geo map settings for flat projection and interactivity
fig_map.update_geos(
    projection_type='equirectangular',  # Flat projection
    landcolor='rgb(217, 217, 217)',
    oceancolor='rgb(233, 245, 248)',
    showocean=True,
    showland=True,
    showcountries=True,
    countrycolor='gray',
    showframe=False,
    showcoastlines=True,
    coastlinecolor='gray',
    bgcolor='rgba(255, 255, 255, 0)',
    # Enable zooming and panning
    resolution=50
)

# Update layout for the map
fig_map.update_layout(
    title_text='Foreign Lobbying Spending by Country (Geographic View)',
    height=600,
    width=1000,
    margin=dict(l=0, r=0, b=0, t=50),
    # Add interactive capabilities
    dragmode='zoom',  # Enable zoom by dragging
    # Add annotations for top 3 spenders for quick reference
    annotations=[
        dict(
            x=df.loc[df['Country'] == 'China', 'lon'].values[0],
            y=df.loc[df['Country'] == 'China', 'lat'].values[0] + 5,
            text="China: $446.1M",
            showarrow=True,
            arrowhead=1,
            ax=0, ay=-30,
            font=dict(size=12, color="black"),
            bgcolor="white",
            bordercolor="black",
            borderwidth=1
        ),
        dict(
            x=df.loc[df['Country'] == 'Japan', 'lon'].values[0],
            y=df.loc[df['Country'] == 'Japan', 'lat'].values[0] + 5,
            text="Japan: $387.3M",
            showarrow=True,
            arrowhead=1,
            ax=0, ay=-30,
            font=dict(size=12, color="black"),
            bgcolor="white",
            bordercolor="black",
            borderwidth=1
        ),
        dict(
            x=df.loc[df['Country'] == 'Liberia', 'lon'].values[0],
            y=df.loc[df['Country'] == 'Liberia', 'lat'].values[0] + 5,
            text="Liberia: $353.1M",
            showarrow=True,
            arrowhead=1,
            ax=0, ay=-30,
            font=dict(size=12, color="black"),
            bgcolor="white",
            bordercolor="black",
            borderwidth=1
        )
    ]
)

# Add interactive hover notes
fig_map.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Arial"
    )
)

# Add zoom and home buttons for enhanced interactivity
fig_map.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=[
                dict(
                    args=[{"geo.projection.scale": 1}],
                    label="Reset Zoom",
                    method="relayout"
                ),
                dict(
                    args=[{"geo.projection.scale": 5, "geo.center.lon": 104.20, "geo.center.lat": 35.86}],
                    label="Zoom to China",
                    method="relayout"
                ),
                dict(
                    args=[{"geo.projection.scale": 5, "geo.center.lon": 138.25, "geo.center.lat": 36.20}],
                    label="Zoom to Japan",
                    method="relayout"
                ),
                dict(
                    args=[{"geo.projection.scale": 5, "geo.center.lon": -9.43, "geo.center.lat": 6.43}],
                    label="Zoom to Liberia",
                    method="relayout"
                )
            ],
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=0,
            yanchor="bottom"
        )
    ]
)

fig_map.show()

# ----- PART 2: Horizontal Bar Chart for Value Comparison -----
fig_bar = go.Figure()

# Sort countries by spending
sorted_df = df.sort_values('Spending_Millions', ascending=True)

# Add horizontal bars
fig_bar.add_trace(go.Bar(
    y=sorted_df['Country'],
    x=sorted_df['Spending_Millions'],
    orientation='h',
    marker=dict(
        color=sorted_df['Spending_Millions'],
        colorscale='Viridis'
    ),
    text=[f"${x:.1f}M" for x in sorted_df['Spending_Millions']],
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>Spending: $%{x:.1f} million<extra></extra>'
))

# Update layout for better readability
fig_bar.update_layout(
    title_text='Foreign Lobbying Spending by Country (Ranked)',
    xaxis_title='Lobbying Spending ($ millions)',
    height=500,
    width=900,
    margin=dict(l=10, r=10, t=50, b=50),
    # Add a note about the data
    annotations=[
        dict(
            x=0,
            y=-0.15,
            xref="paper",
            yref="paper",
            text="Source: Foreign Lobbying Data",
            showarrow=False,
            font=dict(size=12),
            align="left"
        )
    ]
)

fig_bar.show()